<a href="https://colab.research.google.com/github/aekanun2020/DRAFT/blob/main/ANS-Quiz2-PySpark_DataScience_onColab-Rev1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-2.4.1/spark-2.4.1-bin-hadoop2.7.tgz
!tar xf spark-2.4.1-bin-hadoop2.7.tgz
!pip install -q findspark

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.1-bin-hadoop2.7"

In [3]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [4]:
from pyspark.sql import SparkSession

In [5]:
spark = SparkSession.builder \
   .appName("Neural Network Model") \
   .config("spark.executor.memory", "3gb") \
   .getOrCreate()
   
sc = spark.sparkContext

In [6]:
sc

<SparkContext master=local[*] appName=pyspark-shell>

In [7]:
! head -3 testfunnel.csv

head: cannot open 'testfunnel.csv' for reading: No such file or directory


In [8]:
! wget https://storage.googleapis.com/sharewebinar/testfunnel.csv

--2022-02-12 05:25:23--  https://storage.googleapis.com/sharewebinar/testfunnel.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.218.128, 108.177.11.128, 74.125.31.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.218.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1501012 (1.4M) [text/csv]
Saving to: ‘testfunnel.csv’

testfunnel.csv      100%[===================>]   1.43M  --.-KB/s    in 0.02s   

2022-02-12 05:25:23 (89.3 MB/s) - ‘testfunnel.csv’ saved [1501012/1501012]



In [9]:
raw_df = spark.read.option('header','true')\
.option("inferSchema" , "true")\
.format('csv').load('./testfunnel.csv')

In [10]:
raw_df.sample(0.001,seed=12).toPandas().transpose()

,0,1,2,3,4,5,6,7,8,9,10,11,12
source,(direct),(direct),google,google,google,google,(direct),(direct),(direct),google,(direct),google,google
isTrueDirect,None,None,None,None,None,None,True,True,True,None,None,True,None
sourceKeyword,None,None,(not provided),(not provided),(not provided),(not provided),None,None,None,(not provided),None,(not provided),(not provided)
medium,(none),(none),organic,organic,organic,organic,(none),(none),(none),organic,(none),organic,organic
isVideoAd,None,None,None,None,None,None,None,None,None,None,None,None,None
fullVisitorId,4990075466656363861,4990257633001131707,4990697502077570091,4991105026836415438,4991962803971844700,4992499749003799871,4992819126088253676,4993613447705442396,4993613447705442396,4994855995081050813,4995356988620593823,4995584940873803432,4995903322841177843
visitId,1480486355,1485540848,1494612328,1488300417,1475876346,1472235854,1476938325,1482297870,1483430590,1495711333,1481560042,1497463936,1493223582
date,20161129,20170127,20170512,20170228,20161007,20160826,20161019,20161220,20170103,20170525,20161212,20170614,20170426
newVisits,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,1.0,1.0,NaN,1.0
hitReferer,None,https://www.google.nl/,None,None,None,None,None,None,None,https://www.google.ie/,None,None,None


1. ธุรกิจต้องการ ML Model ไปใช้ในการทำนายว่า ลูกค้าที่กำลังเลือกดูสินค้าอยู่ จะซื้อหรือไม่ซื้อสินค้า (รวมถึงพฤติกรรมอื่นๆ ด้วย)

In [11]:
raw_df.count()

9999

2. Data Understanding

In [12]:
len(raw_df.columns)

24

In [13]:
raw_df.describe().toPandas().transpose()

,0,1,2,3,4
summary,count,mean,stddev,min,max
source,9999,None,None,(direct),youtube.com
sourceKeyword,4042,None,None,(not provided),youtube merch
medium,9999,None,None,(none),referral
fullVisitorId,9999,4.5067648825526333E18,1.39995873491150899E18,49898505875812980,4995924842281653133
visitId,9999,1.4850613864552455E9,9020666.664979974,1470106850,1501649190
date,9999,2.016601000310031E7,4694.944823979139,20160801,20170801
newVisits,7269,1.0,0.0,1,1
hitReferer,1333,None,None,http://images.google.co.in/imgres?imgurl=https...,https://www.youtube.com/yt/about/press/
hitType,9999,None,None,EVENT,PAGE


In [14]:
raw_df.printSchema()

root
 |-- source: string (nullable = true)
 |-- isTrueDirect: boolean (nullable = true)
 |-- sourceKeyword: string (nullable = true)
 |-- medium: string (nullable = true)
 |-- isVideoAd: boolean (nullable = true)
 |-- fullVisitorId: long (nullable = true)
 |-- visitId: integer (nullable = true)
 |-- date: integer (nullable = true)
 |-- newVisits: integer (nullable = true)
 |-- hitReferer: string (nullable = true)
 |-- hitType: string (nullable = true)
 |-- hitAction_type: integer (nullable = true)
 |-- hitNumber: integer (nullable = true)
 |-- hitHour: integer (nullable = true)
 |-- hitMin: integer (nullable = true)
 |-- timeMicroSec: integer (nullable = true)
 |-- v2ProductName: string (nullable = true)
 |-- productListName: string (nullable = true)
 |-- isClick: boolean (nullable = true)
 |-- isImpression: boolean (nullable = true)
 |-- sessionQualityDim: integer (nullable = true)
 |-- timeOnScreen: string (nullable = true)
 |-- timeOnSite: integer (nullable = true)
 |-- totalTransac

In [15]:
raw_df.groupBy('isTrueDirect').count().show()

+------------+-----+
|isTrueDirect|count|
+------------+-----+
|        null| 6648|
|        true| 3351|
+------------+-----+



In [16]:
from pyspark.sql import functions as sparkf

In [17]:
selectedCol_df = raw_df\
.drop('isVideoAd').drop('isClick').drop('isImpression')\
.drop('sourceKeyword').drop('hitReferer').drop('timeOnScreen').drop('sessionQualityDim')

In [18]:
selectedCol_df.count()

9999

In [19]:
len(selectedCol_df.columns)

17

3. Data Preparation

In [20]:
from pyspark.sql.types import *

In [21]:
sparkf_NulltoFalse = sparkf.udf(lambda x: 'false' if x is None else x)

In [22]:
selectedCol_df.withColumn('isTrueDirect',\
                          sparkf.col('isTrueDirect').cast(StringType()))\
.withColumn('isTrueDirect',\
           sparkf_NulltoFalse(sparkf.col('isTrueDirect')))\
.groupBy('isTrueDirect').count().show()

+------------+-----+
|isTrueDirect|count|
+------------+-----+
|       false| 6648|
|        true| 3351|
+------------+-----+



In [23]:
sparkf_treatNewVisits = sparkf.udf(lambda x: 'true' if x == '1' else 'false')

In [24]:
selectedCol_df.withColumn('newVisits',\
                          sparkf.col('newVisits').cast(StringType()))\
.withColumn('newVisits',sparkf_treatNewVisits(sparkf.col('newVisits')))\
.groupBy('newVisits').count().show()

+---------+-----+
|newVisits|count|
+---------+-----+
|    false| 2730|
|     true| 7269|
+---------+-----+



In [25]:
sparkf_treatRevenue = sparkf.udf(lambda x: 0 if x == None else x)

In [26]:
selectedCol_df.printSchema()

root
 |-- source: string (nullable = true)
 |-- isTrueDirect: boolean (nullable = true)
 |-- medium: string (nullable = true)
 |-- fullVisitorId: long (nullable = true)
 |-- visitId: integer (nullable = true)
 |-- date: integer (nullable = true)
 |-- newVisits: integer (nullable = true)
 |-- hitType: string (nullable = true)
 |-- hitAction_type: integer (nullable = true)
 |-- hitNumber: integer (nullable = true)
 |-- hitHour: integer (nullable = true)
 |-- hitMin: integer (nullable = true)
 |-- timeMicroSec: integer (nullable = true)
 |-- v2ProductName: string (nullable = true)
 |-- productListName: string (nullable = true)
 |-- timeOnSite: integer (nullable = true)
 |-- totalTransactionRevenue: integer (nullable = true)



In [27]:
sparkf_createLabel = sparkf.udf(lambda x: 'buy' if x>0 else 'notBuy')

In [28]:
crunched_df = selectedCol_df\
.withColumn('isTrueDirect',sparkf.col('isTrueDirect').cast(StringType()))\
.withColumn('isTrueDirect',sparkf_NulltoFalse(sparkf.col('isTrueDirect')))\
.withColumn('newVisits',sparkf.col('newVisits').cast(StringType()))\
.withColumn('newVisits',sparkf_treatNewVisits(sparkf.col('newVisits')))\
.withColumn('totalTransactionRevenue',sparkf_treatRevenue(sparkf.col('totalTransactionRevenue')))\
.withColumn('totalTransactionRevenue',sparkf.col('totalTransactionRevenue').cast(DoubleType()))\
.withColumn('label',sparkf_createLabel(sparkf.col('totalTransactionRevenue')))

In [29]:
crunched_df.printSchema()

root
 |-- source: string (nullable = true)
 |-- isTrueDirect: string (nullable = true)
 |-- medium: string (nullable = true)
 |-- fullVisitorId: long (nullable = true)
 |-- visitId: integer (nullable = true)
 |-- date: integer (nullable = true)
 |-- newVisits: string (nullable = true)
 |-- hitType: string (nullable = true)
 |-- hitAction_type: integer (nullable = true)
 |-- hitNumber: integer (nullable = true)
 |-- hitHour: integer (nullable = true)
 |-- hitMin: integer (nullable = true)
 |-- timeMicroSec: integer (nullable = true)
 |-- v2ProductName: string (nullable = true)
 |-- productListName: string (nullable = true)
 |-- timeOnSite: integer (nullable = true)
 |-- totalTransactionRevenue: double (nullable = true)
 |-- label: string (nullable = true)



In [30]:
crunched_df.describe().toPandas().transpose()

,0,1,2,3,4
summary,count,mean,stddev,min,max
source,9999,None,None,(direct),youtube.com
isTrueDirect,9999,None,None,false,true
medium,9999,None,None,(none),referral
fullVisitorId,9999,4.5067648825526333E18,1.39995873491150899E18,49898505875812980,4995924842281653133
visitId,9999,1.4850613864552455E9,9020666.664979974,1470106850,1501649190
date,9999,2.016601000310031E7,4694.944823979139,20160801,20170801
newVisits,9999,None,None,false,true
hitType,9999,None,None,EVENT,PAGE
hitAction_type,9999,0.13831383138313832,0.6864877640641197,0,6


In [31]:
crunched_df.dropna().count()

8958

In [32]:
final_df = crunched_df.dropna()

In [33]:
final_df.describe().toPandas().transpose()

,0,1,2,3,4
summary,count,mean,stddev,min,max
source,8958,None,None,(direct),youtube.com
isTrueDirect,8958,None,None,false,true
medium,8958,None,None,(none),referral
fullVisitorId,8958,4.5213872161084938E18,1.37948897535059482E18,49898679051871719,4995924842281653133
visitId,8958,1.484833213452668E9,8954199.880686603,1470106850,1501649190
date,8958,2.01658726431123E7,4700.377098126274,20160801,20170801
newVisits,8958,None,None,false,true
hitType,8958,None,None,EVENT,PAGE
hitAction_type,8958,0.15438713998660417,0.7235710402472797,0,6


In [34]:
final_df.groupBy('label').count().show()

+------+-----+
| label|count|
+------+-----+
|   buy|  707|
|notBuy| 8251|
+------+-----+



4. Modeling

In [35]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder

In [36]:
final_df.columns

['source',
 'isTrueDirect',
 'medium',
 'fullVisitorId',
 'visitId',
 'date',
 'newVisits',
 'hitType',
 'hitAction_type',
 'hitNumber',
 'hitHour',
 'hitMin',
 'timeMicroSec',
 'v2ProductName',
 'productListName',
 'timeOnSite',
 'totalTransactionRevenue',
 'label']

In [37]:
final_df.filter(sparkf.col('visitId') == '1470106850')\
.orderBy('hitNumber')\
.select(['fullVisitorId','visitId','hitHour','hitMin','v2ProductName','hitAction_type','label'])\
.show(100)

+-------------------+----------+-------+------+--------------------+--------------+-----+
|      fullVisitorId|   visitId|hitHour|hitMin|       v2ProductName|hitAction_type|label|
+-------------------+----------+-------+------+--------------------+--------------+-----+
|4993485206334150199|1470106850|     20|     1|Women's YouTube S...|             0|  buy|
|4993485206334150199|1470106850|     20|     1| YouTube Trucker Hat|             0|  buy|
|4993485206334150199|1470106850|     20|     1|YouTube Women's R...|             0|  buy|
|4993485206334150199|1470106850|     20|     1|YouTube Women's S...|             0|  buy|
|4993485206334150199|1470106850|     20|     1|22 oz YouTube Bot...|             0|  buy|
|4993485206334150199|1470106850|     20|     1|YouTube Men's Sho...|             0|  buy|
|4993485206334150199|1470106850|     20|     1|YouTube Men's Lon...|             0|  buy|
|4993485206334150199|1470106850|     20|     1|YouTube Men's 3/4...|             0|  buy|
|499348520

In [38]:
final_df.filter(sparkf.col('label') == 'buy').orderBy(['visitId','timeMicroSec']).limit(10).toPandas().transpose()

,0,1,2,3,4,5,6,7,8,9
source,youtube.com,youtube.com,youtube.com,youtube.com,youtube.com,youtube.com,youtube.com,youtube.com,youtube.com,youtube.com
isTrueDirect,false,false,false,false,false,false,false,false,false,false
medium,referral,referral,referral,referral,referral,referral,referral,referral,referral,referral
fullVisitorId,4993485206334150199,4993485206334150199,4993485206334150199,4993485206334150199,4993485206334150199,4993485206334150199,4993485206334150199,4993485206334150199,4993485206334150199,4993485206334150199
visitId,1470106850,1470106850,1470106850,1470106850,1470106850,1470106850,1470106850,1470106850,1470106850,1470106850
date,20160801,20160801,20160801,20160801,20160801,20160801,20160801,20160801,20160801,20160801
newVisits,true,true,true,true,true,true,true,true,true,true
hitType,PAGE,PAGE,PAGE,PAGE,PAGE,PAGE,PAGE,PAGE,PAGE,PAGE
hitAction_type,0,0,0,0,0,0,0,0,0,0
hitNumber,2,2,2,2,2,2,2,2,2,2


In [39]:
final_df.orderBy(['visitId','timeMicroSec']).limit(10).toPandas().transpose()

,0,1,2,3,4,5,6,7,8,9
source,youtube.com,youtube.com,youtube.com,youtube.com,youtube.com,youtube.com,youtube.com,youtube.com,youtube.com,youtube.com
isTrueDirect,false,false,false,false,false,false,false,false,false,false
medium,referral,referral,referral,referral,referral,referral,referral,referral,referral,referral
fullVisitorId,4993485206334150199,4993485206334150199,4993485206334150199,4993485206334150199,4993485206334150199,4993485206334150199,4993485206334150199,4993485206334150199,4993485206334150199,4993485206334150199
visitId,1470106850,1470106850,1470106850,1470106850,1470106850,1470106850,1470106850,1470106850,1470106850,1470106850
date,20160801,20160801,20160801,20160801,20160801,20160801,20160801,20160801,20160801,20160801
newVisits,true,true,true,true,true,true,true,true,true,true
hitType,PAGE,PAGE,PAGE,PAGE,PAGE,PAGE,PAGE,PAGE,PAGE,PAGE
hitAction_type,0,0,0,0,0,0,0,0,0,0
hitNumber,2,2,2,2,2,2,2,2,2,2


In [40]:
sourceIndexer = StringIndexer(inputCol='source', outputCol='sourceIndexed',
    handleInvalid='keep')
sourceOneHotEncoder = OneHotEncoder(inputCol='sourceIndexed', outputCol='sourceVec')

In [41]:
isTrueDirectIndexer = StringIndexer(inputCol='isTrueDirect', outputCol='isTrueDirectIndexed',
    handleInvalid='keep')
isTrueDirectOneHotEncoder = OneHotEncoder(inputCol='isTrueDirectIndexed', outputCol='isTrueDirectVec')

In [42]:
mediumIndexer = StringIndexer(inputCol='medium', outputCol='mediumIndexed',
    handleInvalid='keep')
mediumOneHotEncoder = OneHotEncoder(inputCol='mediumIndexed', outputCol='mediumVec')

In [43]:
newVisitsIndexer = StringIndexer(inputCol='newVisits', outputCol='newVisitsIndexed',
    handleInvalid='keep')
newVisitsOneHotEncoder = OneHotEncoder(inputCol='newVisitsIndexed', outputCol='newVisitsVec')

In [44]:
hitTypeIndexer = StringIndexer(inputCol='hitType', outputCol='hitTypeIndexed',
    handleInvalid='keep')
hitTypeOneHotEncoder = OneHotEncoder(inputCol='hitTypeIndexed', outputCol='hitTypeVec')

In [45]:
v2ProductNameIndexer = StringIndexer(inputCol='v2ProductName', outputCol='v2ProductNameIndexed',
    handleInvalid='keep')
v2ProductNameOneHotEncoder = OneHotEncoder(inputCol='v2ProductNameIndexed', outputCol='v2ProductNameVec')

In [46]:
productListNameIndexer = StringIndexer(inputCol='productListName', outputCol='productListNameIndexed',
    handleInvalid='keep')
productListNameOneHotEncoder = OneHotEncoder(inputCol='productListNameIndexed', outputCol='productListNameVec')

In [47]:
labelIndexer = StringIndexer(inputCol='label', outputCol='labelIndexed',
    handleInvalid='keep')
#labelOneHotEncoder = OneHotEncoder(inputCol='labelIndexed', outputCol='labelVec')

In [48]:
from pyspark.mllib.linalg import Vectors

In [49]:
from pyspark.ml.feature import VectorAssembler

In [50]:
from pyspark.ml import Pipeline

In [51]:
featureAssembler = VectorAssembler(inputCols=['sourceVec'\
                                             ,'isTrueDirectVec'\
                                             ,'mediumVec'\
                                             ,'newVisitsVec'\
                                             ,'hitTypeVec'\
                                             ,'v2ProductNameVec'\
                                             ,'productListNameVec'\
                                             ,'hitNumber'\
                                             ,'timeMicroSec'], outputCol='features')

In [52]:
from pyspark.ml.classification import DecisionTreeClassifier

In [53]:
algo = DecisionTreeClassifier(labelCol='labelIndexed',featuresCol='features')

In [54]:
pipeline_train = Pipeline().setStages([sourceIndexer,sourceOneHotEncoder,isTrueDirectIndexer,isTrueDirectOneHotEncoder\
                     ,mediumIndexer,mediumOneHotEncoder,newVisitsIndexer,newVisitsOneHotEncoder\
                     ,hitTypeIndexer,hitTypeOneHotEncoder,v2ProductNameIndexer,v2ProductNameOneHotEncoder\
                     ,productListNameIndexer,productListNameOneHotEncoder\
                                       ,labelIndexer\
                                       ,featureAssembler,algo])

In [55]:
training_df, testing_df = final_df.randomSplit(weights = [0.80, 0.20], seed = 13)

In [56]:
model = pipeline_train.fit(training_df)

In [57]:
model

PipelineModel_575c7a801035

5. Model Evaluation

In [58]:
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator

In [59]:
result_df = model.transform(testing_df)
evaluator_RF = MulticlassClassificationEvaluator(predictionCol="prediction",\
                                              labelCol="labelIndexed", metricName='accuracy')
print('evaluation','====>',evaluator_RF.evaluate(result_df)*100)

evaluation ====> 93.78042184964845


In [60]:
result_df

DataFrame[source: string, isTrueDirect: string, medium: string, fullVisitorId: bigint, visitId: int, date: int, newVisits: string, hitType: string, hitAction_type: int, hitNumber: int, hitHour: int, hitMin: int, timeMicroSec: int, v2ProductName: string, productListName: string, timeOnSite: int, totalTransactionRevenue: double, label: string, sourceIndexed: double, sourceVec: vector, isTrueDirectIndexed: double, isTrueDirectVec: vector, mediumIndexed: double, mediumVec: vector, newVisitsIndexed: double, newVisitsVec: vector, hitTypeIndexed: double, hitTypeVec: vector, v2ProductNameIndexed: double, v2ProductNameVec: vector, productListNameIndexed: double, productListNameVec: vector, labelIndexed: double, features: vector, rawPrediction: vector, probability: vector, prediction: double]

In [61]:
result_df.groupBy('labelIndexed').count().show()

+------------+-----+
|labelIndexed|count|
+------------+-----+
|         0.0| 1701|
|         1.0|  148|
+------------+-----+



In [62]:
result_df.filter(sparkf.col('labelIndexed') == sparkf.col('prediction'))\
.select(['labelIndexed','prediction']).groupBy('labelIndexed').count().show()

+------------+-----+
|labelIndexed|count|
+------------+-----+
|         0.0| 1693|
|         1.0|   41|
+------------+-----+



In [63]:
result_df.filter(sparkf.col('labelIndexed') != sparkf.col('prediction'))\
.select(['labelIndexed','prediction']).groupBy('labelIndexed').count().show()

+------------+-----+
|labelIndexed|count|
+------------+-----+
|         0.0|    8|
|         1.0|  107|
+------------+-----+



In [64]:
Recall = 2782/14015

In [65]:
Recall

0.19850160542276132

In [66]:
final_df

DataFrame[source: string, isTrueDirect: string, medium: string, fullVisitorId: bigint, visitId: int, date: int, newVisits: string, hitType: string, hitAction_type: int, hitNumber: int, hitHour: int, hitMin: int, timeMicroSec: int, v2ProductName: string, productListName: string, timeOnSite: int, totalTransactionRevenue: double, label: string]

In [67]:
final_df.groupBy('medium','hitType')\
        .agg(\
             sparkf.count(sparkf.col('hitType'))\
             ,sparkf.avg(sparkf.col('hitMin'))\
        ).show()

+---------+-------+--------------+------------------+
|   medium|hitType|count(hitType)|       avg(hitMin)|
+---------+-------+--------------+------------------+
|   (none)|  EVENT|           152|30.894736842105264|
|      cpm|   PAGE|           284|42.485915492957744|
|  organic|   PAGE|          3538| 31.51526286037309|
|  organic|  EVENT|           134|29.380597014925375|
|      cpc|  EVENT|             3| 7.333333333333333|
|affiliate|   PAGE|            72|              19.5|
|   (none)|   PAGE|          4226| 32.47113109323237|
|      cpc|   PAGE|            87|20.264367816091955|
| referral|   PAGE|           437| 33.70938215102975|
| referral|  EVENT|            12|25.333333333333332|
|      cpm|  EVENT|            13| 42.76923076923077|
+---------+-------+--------------+------------------+



In [68]:
num_dim = 'hitMin'
quantileF = sparkf.expr('percentile_approx('+num_dim+', array(0.25,0.5,0.75))')

In [69]:

final_df.groupBy('medium','hitType')\
        .agg(\
             sparkf.count(sparkf.col('hitType'))\
             ,sparkf.min(sparkf.col('hitMin')).alias('min')\
             ,sparkf.max(sparkf.col('hitMin')).alias('max')\
             ,sparkf.avg(sparkf.col('hitMin')).alias('sampleMean')\
             ,quantileF.alias('quantile')\
             ,sparkf.stddev(sparkf.col('hitMin')).alias('stddev')\
        ).show()

+---------+-------+--------------+---+---+------------------+------------+------------------+
|   medium|hitType|count(hitType)|min|max|        sampleMean|    quantile|            stddev|
+---------+-------+--------------+---+---+------------------+------------+------------------+
|   (none)|  EVENT|           152|  0| 59|30.894736842105264|[19, 28, 46]|15.188073874252758|
|      cpm|   PAGE|           284|  5| 57|42.485915492957744|[40, 45, 55]| 14.01878392345356|
|  organic|   PAGE|          3538|  0| 59| 31.51526286037309|[22, 35, 43]|15.387389280992426|
|  organic|  EVENT|           134|  4| 59|29.380597014925375|[23, 25, 40]|13.234038350762958|
|      cpc|  EVENT|             3|  7|  8| 7.333333333333333|   [7, 7, 8]|0.5773502691896258|
|affiliate|   PAGE|            72|  8| 32|              19.5|[18, 19, 20]|7.0190886612011685|
|   (none)|   PAGE|          4226|  0| 59| 32.47113109323237|[20, 32, 47]| 16.56094341703807|
|      cpc|   PAGE|            87|  7| 53|20.264367816091955